In [26]:
import pyarrow.parquet as pq
pq.read_metadata('/home/junjie/test/reference/emb_coco_train_ref/metadata/metadata_0.parquet')

  created_by: parquet-cpp-arrow version 7.0.0
  num_columns: 1
  num_rows: 118287
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 1598

In [27]:
import pandas as pd
pd.read_parquet('/home/junjie/test/reference/emb_coco_train_ref/metadata/metadata_0.parquet', engine='pyarrow')

,image_path
0,/ssd/mlrom/Data/coco2017/train2017/00000000000...
1,/ssd/mlrom/Data/coco2017/train2017/00000000002...
2,/ssd/mlrom/Data/coco2017/train2017/00000000003...
3,/ssd/mlrom/Data/coco2017/train2017/00000000003...
4,/ssd/mlrom/Data/coco2017/train2017/00000000003...
...,...
118282,/ssd/mlrom/Data/coco2017/train2017/00000058190...
118283,/ssd/mlrom/Data/coco2017/train2017/00000058190...
118284,/ssd/mlrom/Data/coco2017/train2017/00000058191...
118285,/ssd/mlrom/Data/coco2017/train2017/00000058192...


In [1]:
from embedding_reader import EmbeddingReader

embedding_reader = EmbeddingReader(
    embeddings_folder="/home/junjie/test/reference/emb_coco_train_ref_l14/img_emb",
    file_format="parquet_npy",
    #file_format ="npy",
    embedding_column="embedding",
    metadata_folder="/home/junjie/test/reference/emb_coco_train_ref_l14/metadata",
    meta_columns=["image_path"],
)

nb_vectors = embedding_reader.count
vec_dim = embedding_reader.dimension
print("There are {} embeddings of dim {}".format(nb_vectors, vec_dim))

for embs, meta in embedding_reader(batch_size=10**6, start=0, end=embedding_reader.count):
    print(embs.shape)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5184.55it/s]


There are 118287 embeddings of dim 768


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.96it/s]

(118287, 768)


In [22]:
import pandas as pd
l14_weight = pd.read_csv('/home/junjie/git/clip-retrieval-aizip/classifier/l14_weight.csv', sep=' ', header=None)
l14_bias = pd.read_csv('/home/junjie/git/clip-retrieval-aizip/classifier/l14_bias.csv', sep=' ', header=None)
print(l14_weight.shape)
print(l14_bias.shape)
print(l14_bias)
print(l14_bias.to_numpy().reshape((-1,1)))

(2, 768)
(2, 1)
          0
0  0.039118
1 -0.039118
[[ 0.03911775]
 [-0.03911841]]


In [27]:
# Short test
import numpy as np
i = 80
res_tmp = np.dot(l14_weight, embs[i])
print(res_tmp.shape)
print(res_tmp)
print("-----------")
res_reshape = res_tmp.reshape((-1,1))
print(res_reshape.shape)
print(res_reshape)
print("-----------")
res = np.add(res_reshape,l14_bias.to_numpy())
print(res.shape)
print(res)
print("-----------")
print("Result 0: {}\nResult 1:{}".format(res[0][0], res[1][0]))
print("-----------")
res_fi = 0 if res[0][0]>res[1][0] else 1
print("Final result: {}".format(res_fi))
meta["image_path"][i]
print("-----------")
print("type after reshape: {}, type of bias: {}, type after add: {}".format(type(res_reshape), type(l14_bias.to_numpy()), type(res)))
print("res in list: {}".format(res.tolist()))

(2,)
[-0.01132164  0.02359465]
-----------
(2, 1)
[[-0.01132164]
 [ 0.02359465]]
-----------
(2, 1)
[[ 0.02779611]
 [-0.01552375]]
-----------
Result 0: 0.027796105390516468
Result 1:-0.015523751798773022
-----------
Final result: 0
-----------
type after reshape: <class 'numpy.ndarray'>, type of bias: <class 'numpy.ndarray'>, type after add: <class 'numpy.ndarray'>
res in list: [[0.027796105390516468], [-0.015523751798773022]]


In [30]:
# Run classifier over all emb

total_results = []
for idx, emb in enumerate(embs):
    single_dict = {}
    res_tmp = np.add(np.dot(l14_weight, emb).reshape(-1,1),l14_bias.to_numpy())
    single_dict["classify_tmp_res"] = res_tmp.tolist()
    single_dict["classify_res"] = 0 if res_tmp[0][0]>res_tmp[1][0] else 1
    single_dict["image_path"] = meta["image_path"][idx]
    total_results.append(single_dict)

In [31]:
print(len(total_results))
print(total_results[0])

118287
{'classify_tmp_res': [[0.14625618030281023], [-0.1379117523705332]], 'classify_res': 0, 'image_path': '/ssd/mlrom/Data/coco2017/train2017/000000000009.jpg'}


In [32]:
import pandas as pd

write_df = pd.DataFrame.from_dict(total_results)
write_df.to_parquet("/home/junjie/test/reference/emb_coco_train_ref_l14/classifier.parquet")

#import pyarrow as pa
#table = pa.Table.from_pandas(write_df)